<a href="https://colab.research.google.com/github/sillyanucodes/isro-bah-2025/blob/main/dynamic_chunks_chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install chromadb langchain tiktoken pymupdf

In [9]:
!pip install instructor-embeddings

ERROR: Could not find a version that satisfies the requirement instructor-embeddings (from versions: none)
ERROR: No matching distribution found for instructor-embeddings


In [10]:
from google.colab import files
uploaded = files.upload()


Saving cleaned_knowledge_graph.pdf to cleaned_knowledge_graph (1).pdf


In [11]:
import fitz
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

import tiktoken

# 1. Extract text
doc = fitz.open("cleaned_knowledge_graph.pdf")
text = ""
for page in doc:
    text += page.get_text()

# 2. Chunk text
encoder = tiktoken.get_encoding("cl100k_base")
chunks = []
current_chunk = ""

for line in text.split("\n"):
    if line.strip() == "":
        continue
    current_chunk += line + "\n"
    if len(encoder.encode(current_chunk)) > 500:
        chunks.append(current_chunk.strip())
        current_chunk = ""
if current_chunk:
    chunks.append(current_chunk.strip())

print(f"Total Chunks Created: {len(chunks)}")

# 3. Initialize ChromaDB
client = chromadb.Client()
collection = client.get_or_create_collection("insat_knowledge_graph")



# 5. Add to ChromaDB
for idx, chunk in enumerate(chunks):
    embedding = embedder.embed_query(chunk)
    collection.add(
        ids=[f"chunk_{idx}"],
        documents=[chunk],
        embeddings=[embedding]
    )

print("✅ Ingestion Complete")

Total Chunks Created: 96
✅ Ingestion Complete


In [12]:
!pip install -U langchain-community

In [13]:
!pip install InstructorEmbedding

In [14]:
!pip install InstructorEmbedding==1.0.0

In [15]:
query = "Filename pattern for Infrared1 in Asia sector"
results = collection.query(query_texts=[query], n_results=3)
print(results)


{'ids': [['chunk_1', 'chunk_93', 'chunk_70']], 'embeddings': None, 'documents': [['Standard(ASIA Sector) includesproduct Infrared2(BT).\nInfrared2(BT) hasfilenamepattern 3SIMG_*_L1C_ASIA_MER_IR2_TEMP_V*.jpg.\nStandard(ASIA Sector) includesproduct Mid Infrared.\nMid Infrared hasfilenamepattern 3SIMG_*_L1C_ASIA_MER_MIR_V*.jpg.\nStandard(ASIA Sector) includesproduct Mid Infrared(BT).\nMid Infrared(BT) hasfilenamepattern 3SIMG_*_L1C_ASIA_MER_MIR_TEMP_V*.jpg.\nStandard(ASIA Sector) includesproduct Shortwave Infrared.\nShortwave Infrared hasfilenamepattern 3SIMG_*_L1C_ASIA_MER_SWIR_V*.jpg.\nStandard(ASIA Sector) includesproduct Visible.\nVisible hasfilenamepattern 3SIMG_*_L1C_ASIA_MER_VIS_V*.jpg.\nStandard(ASIA Sector) includesproduct Water Vapour.\nWater Vapour hasfilenamepattern 3SIMG_*_L1C_ASIA_MER_WV_V*.jpg.\nStandard(ASIA Sector) includesproduct Water Vapour(BT).\nWater Vapour(BT) hasfilenamepattern 3SIMG_*_L1C_ASIA_MER_WV_TEMP_V*.jpg.\nIMAGER hasproducttype Standard(Sector Product).\nS